<img src="ost_logo.png" width="240" height="240" align="right"/>
<div style="text-align: left"> <b> Machine Learning </b> <br> MSE FTP MachLe <br> 
<a href="mailto:christoph.wuersch@ost.ch"> Christoph Würsch </a> </div>

## Temporal Difference Learning Gridworld Demo

Source: [GridWorld](https://cs.stanford.edu/people/karpathy/reinforcejs/gridworld_td.html)

## Setup

This is a toy environment called **Gridworld** that is often used as a toy model in the Reinforcement Learning literature. In this particular case:

- **State space**: GridWorld has 10x10 = 100 distinct states. The start state is the top left cell. The gray cells are walls and cannot be moved to.
- **Actions**: The agent can choose from up to 4 actions to move around. In this example
- **Environment Dynamics**: GridWorld is deterministic, leading to the same new state given each state and action
Rewards: The agent receives +1 reward when it is in the center square (the one that shows R 1.0), and -1 reward in a few states (R -1.0 is shown for these). The state with +1.0 reward is the goal state and resets the agent back to start.



In other words, this is a deterministic, **finite Markov Decision Process (MDP)** and as always the goal is to find an agent policy (shown here by arrows) that maximizes the future discounted reward. My favorite part is letting the agent find the optimal path, then suddenly change the reward of some cell with the slider and watch it struggle to reroute its policy.

**Interface**. The color of the cells (initially all white) shows the current estimate of the Value (discounted reward) of that state, with the current policy. Note that you can select any cell and change its reward with the Cell reward slider.

## Temporal Difference Learning

TD methods (for finite MDPs) are covered very nicely in Richard Sutton's Free [Online Book](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf) on Reinforcement Learning, in this particular case Chapter 6.

Briefly, the core idea is to estimate the action value function $Q^\pi(s,a)$, which is the expected discounted reward obtained by the agent by taking action \(a\) in state \(s\) and then following some particular policy $\pi(a \mid s)$:

$$ Q^\pi (s,a) = E_\pi \{ r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \ldots \mid s_t = s, a_t = a \} $$

The expectation above is really over two **stochastic sources**: 
1. the *environment* and 
2. the *agent policy*

which in the general case is also stochastic. 

Unlike *Dynamic Programming*, **Temporal Difference Learning** estimates the value functions from the point of view of an agent who is interacting with the environment, collecting experience about its dynamics and adjusting its policy online. That is, the agent's interaction with the environment (which forms our training set) is a long sequence of $(s_t, a_t, r_t, s_{t+1}, a_{t+1}, r_{t+1}, s_{t+2}, \ldots )$, indexed by $t$ (time):

<img src="sarsa.png" alt="drawing" width="400" align="center"/>

However, unlike a standard Machine Learning setting the agent picks the actions, and **hence influences its own training set**. Fun! The core idea is to notice that the $Q$ function satisfies the *Bellman equation*, which is a recurrence relation that relates the Q function at one action node $(s_t, a_t)$ to the next one $(s_{t+1}, a_{t+1})$. In particular, looking at the above diagram the agent:

- The agent picked some action $a_t$ in state $s_t$
- The environment responded with some reward $r_t$ and new state $s_{t+1}$
- The agent then picks some new action $a_{t+1}$ from its current policy $\pi$.

We can explicitly write out the expect reward of this behavior and express $Q$ **recursively** based on itself:

$$ Q(s,a) = \sum_{s'} \mathcal{P}_{ss'}^a \left[ \mathcal{R}_{ss'}^a + \gamma \sum_{a'} \pi(s',a') Q(s',a') \right] $$

## On-policy Learning: SARSA =  $(s,a,r,s',a')$
Where we see the two sources of randomness explicitly summed over (first over the next state, and then over the agent's current policy). 
- Since this equation is expected to hold, our strategy is to initialize $Q$ with some numbers (e.g. all zeros) and then turn this recurrence relation into an update. 
- Of course, we don't have access to the environment dynamics $\mathcal{P}_{ss'}$, but we can base the update on the agent's experience from interacting with the environment, which is at least a sample from this unknown distribution. 
- Notice that we do have access to the policy $\pi$ so we could in principle evaluate the second sum exactly, but in practice it is simpler (especially if your actions were continuous) to sample this part as well, giving rise to the **SARSA (short for s,a,r,s',a', get it?) algorithm**:

$$ Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ \underbrace{r_t + \gamma Q(s_{t+1}, a_{t+1})}_{target} - \underbrace{Q(s_t, a_t)}_{current} \right] $$


## Update-Rule = Online stochastic gradient descent for Bellman-equation


We can also interpret SARSA as weighted average between the current estimate and the future estimate (expectation value).

$$ Q(s_t, a_t) \leftarrow (1-\alpha) \cdot \underbrace{Q(s_t, a_t)}_{current} + \alpha \underbrace{ \left[  r_t + \gamma Q(s_{t+1}, a_{t+1})\right]}_{target} $$

Here the parameter $\alpha$ is the *learning rate*, and the quantity inside the bracket is called the **TD Error**. 

- In other words: the idea is to interact with the environment by starting with some initial $Q$, using some policy $\pi$, and keeping track of a chain of $(s,a,r,...)$.
- We then simply treat this as training data, and use **online stochastic gradient descent to minimize the loss function**, which in this case is **the Bellman recurrence relation**. 


## Policy Iteration

- Each time we perform the update to the $Q$ function, we can also update our policy to be greedy with respect to our new belief about $Q$. That is, in each state the policy becomes to take the action that maximizes $Q$. This approach of starting with some value function and policy and iteratively updating one based on the other is the policy iteration scheme described in Sutton's book:

<img src="policyiter.png" alt="drawing" width="300" align="center"/>

## Q-Learning (off-policy)

Now, SARSA is called an on-policy method because it's evaluating the $Q$ function for a particular policy. It turns out that if you're interested in control rather than estimating $Q$ for some policy, in practice there is an update that works much better. It's called **Q-Learning** and it has the form:

$$ Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_t + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t) \right] $$

We can also interpret Q-Learning as weighted average between the current estimate and the future *maximum* expectation value.

$$ Q(s_t, a_t) \leftarrow  (1-\alpha) \cdot Q(s_t, a_t) + \alpha \cdot  \left[ r_t + \gamma \max_a Q(s_{t+1}, a)\right] $$

This is an **off-policy update because the behavior policy does not match the policy whose Q function is being approximated**, in this case the optimal action-value function $Q^*$. Intuitively, the update looks optimistic, since it updates the $Q$ function based on its estimate of the value of the best action it can take at state $s_{t+1}$, not based on the action it happened to sample with its current behavior policy. **With this Gridworld demo as well, the Q-Learning update converges much faster than SARSA.**

## Exploration and $\varepsilon$-greedy policy

The last necessary component to get TD Learning to work well is to explicitly ensure **some amount of exploration**.
- If the agent always follows its current policy, the danger is that it can get stuck exploiting, somewhat similar to getting stuck in local minima during optimization. 
- One common and perhaps simplest way to ensure some exploration (and make all converge proofs work) is to make the agent's policy **epsilon-greedy**. 
- That is, with probability $\varepsilon$ the agent takes a random action, and the remainder of the time it follows its current policy. Some usual settings for this parameter are $\varepsilon \approx 0.1, 0.2,\dots$ or so, and this is usually annealed over the duration of the trianing time to very small numbers (e.g. 0.05 or 0.01, etc).

## Planning. 

- TD methods are by default called **model-free**, because they do not need to estimate the environment model. 
- That is, we do not need to know the **transition probabilities** $\mathcal{P}_{ss'}$, or the **rewards** assigned by the environment to the agent $\mathcal{R}$. 
- We only *observe samples of these values* and distil their sufficient statistics in the action value function $Q$.

However, a model of the environment can still prove extremely useful. The basic idea in planning is that we will explicitly maintain a model of how the environment works (trained again, from observation data). 
- That is, what states and actions lead to what other states, and how the rewards are assigned. In other words, based on our training data trace $s_t, a_t, r_t, s_{t+1}, a_{t+1}, r_{t+1}, s_{t+2}, \ldots $ , the agent keeps track of the environment model:

$$ \mathrm{Model}(s,a) \leftarrow s', r $$

In this Gridworld demo for example, it's just a simple array lookup for all pairs of $(s,a)$. The fact that this Gridworld is deterministic helps, because these are all just fixed, unchanging values that are easy to observe once and then remember. This model is very useful because it can help us hallucinate experiences, and perform updates based on these fake experiences.